<a href="https://colab.research.google.com/github/HuyenNguyenHelen/MyTREC_2021/blob/main/BERT_Innitial_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers rank_bm25

     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 2.5MB 14.1MB/s 
     |████████████████████████████████| 1.2MB 47.0MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 901kB 46.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=16b8f27468b660d120d753beea9226c1704427e4598b2ede1245c508338e7548
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


# Documents and Query processing

### Documents

In [13]:
import pandas as pd
path = r"/content/Text_New_Summary_OR_NER-BERN_Query2016_gene-disease-drug_2016(1).csv"
with open(path, 'r', encoding = 'utf-8' ) as f:
    data = pd.read_csv(f, header = None, delimiter='\t')
data.columns = ['queryID','docID', 'score', 'concat_str']
data.head()

# column_names = [name.split(':')[0].replace('\'', '') for name in data.columns]
# column_names = [name.split(':')[0].replace(' ', '') for name in column_names]
# data.columns = column_names
# print(data.columns )

# # Creating new data with some txt fields
# new_data = data[['brief_title', 'official_title', 'brief_summary_text_block', 'concat_string']]
# new_data['brief_summary_text_block'] = new_data['brief_summary_text_block'].apply(lambda x: x.split(':')[1].replace("'",''))
# new_data['brief_summary_text_block'].head()

,queryID,docID,score,concat_str
0,1,4391998,16.155838,avail urea nitrogencreatinin gastrointestin bl...
1,1,4529650,15.569978,duoden adenocarcinoma melena duoden melena mic...
2,1,4798841,15.293642,preoper cavern hemangioma present melena wirel...
3,1,3999026,15.052819,anemia melena pyeloduoden fistula stoneassoci ...
4,1,3113951,14.672893,ultrasonograph cow abomas lymphosarcoma descri...


In [14]:
doc_txt = data['concat_str'].values.tolist()

### Query

In [ ]:
# Raw query
'''
# Loading the Query [2016/2021]
## Query 2016
import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse(r"/content/topics2016.xml")
root = tree.getroot()

queryID, note, description, summary=[],[],[], []

 
i=0
while i<len(root):
    queryID.append(i+1)
    for j in range (len(root[i])):
        if root[i][j].tag == 'note':
            note.append(root[i][j].text.replace('\n',''))
        if root[i][j].tag == 'summary':
            summary.append(root[i][j].text)
        if root[i][j].tag == 'description':
            description.append(root[i][j].text)
  
    i+=1
    
queries = pd.DataFrame(zip(queryID, note, description, summary), columns =['queryID', 'note', 'description', 'summary'] )
queries.head()
'''

,queryID,note,description,summary
0,1,78 M w/ pmh of CABG in early [**Month (only...,78 M transferred to nursing home for rehab aft...,A 78 year old male presents with frequent stoo...
1,2,Ms [**Known patient lastname 241**] is a [*...,An elderly female with past medical history of...,An elderly female with past medical history of...
2,3,Pt is a 75F with a PMHx significant for sev...,"A 75F with a PMHx significant for severe PVD, ...",A 75F found to be hypoglycemic with hypotensio...
3,4,The patient is an 87 yo woman with h/o oste...,"An 87 yo woman with h/o osteoporosis, multiple...","An 87 yo woman with h/o osteoporosis, DM2, dem..."
4,5,"An 82 M with COPD, s/p bioprosthetic AVR fo...",An 82 man with chronic obstructive pulmonary d...,An 82 man with multiple chronic conditions and...


In [17]:
# Processed query 2016
path = r"/content/NER-BERN_Query2021_gene-disease-drug.csv"
with open(path, 'r', encoding = 'utf-8' ) as f:
    query = pd.read_csv(f)
query['NERgdd'] = query['NERgdd'].apply(labda x: ''.join([i for i in x.split(',')]))
query.head()


,Unnamed: 0,QueryID,Age,Gender,Main_text,Main_text_NER,NERgdd
0,0,1,45,Male,Patient is a 45-year-old man with a history of...,{'disease': 'anaplastic astrocytoma of the spi...,"anaplastic astrocytoma of the spine,temozolomide"
1,1,2,48,Male,"48 M with a h/o HTN hyperlipidemia, bicuspid a...","{'disease': 'HTN hyperlipidemia', 'gene': 'EF'...","HTN hyperlipidemia,EF,Lasix"
2,2,3,32,Female,A 32 yo woman who presents following a severe ...,"{'disease': 'intraventricular hemorrhage', 'sp...",intraventricular hemorrhage
3,3,4,44,Female,"This is a 44 year old female with PMH of PCOS,...","{'disease': 'PMH', 'gene': 'kappa light chain'}","PMH,kappa light chain"
4,4,5,74,Male,"74M hx of CAD s/p CABG, EF 60% prior CVA (no r...","{'disease': 'CAD', 'species': 'patient'}",CAD


In [23]:
# Processing query

IndentationError: ignored

# Question-Answering using Simple Wikipedia Index

This examples demonstrates the setup for Query / Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

For semantic search, we use SentenceTransformer('msmarco-distilbert-base-v2') and retrieve
100 potentially passages that answer the input query.

Next, we use a more powerful CrossEncoder (cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')) that
scores the query and all retrieved passages for their relevancy. The cross-encoder is neccessary to filter out certain noise
that might be retrieved from the semantic search step.


In [19]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-MiniLM-L-6-v3'
bi_encoder = SentenceTransformer(model_name)
top_k = 100  # Number of passages we want to retrieve with the bi-encoder

# The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
# Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
corpus_embeddings = bi_encoder.encode(doc_txt, convert_to_tensor=True, show_progress_bar=True)
print(corpus_embeddings)


tensor([[ 0.3816,  0.0557,  0.1371,  ..., -0.0199,  0.4855,  0.2420],
        [ 0.0024, -0.2993,  0.0318,  ..., -0.3888, -0.2327,  0.1130],
        [ 0.0950,  0.0926,  0.4330,  ..., -0.4781,  0.3871,  0.3569],
        ...,
        [-0.4757, -0.4969, -0.2152,  ..., -0.0865,  0.4372, -0.4299],
        [-0.1003, -0.0306,  0.1446,  ..., -0.1608,  0.4007,  0.2513],
        [-0.3700, -0.0859, -0.0194,  ..., -0.1801, -0.0886,  0.5117]],
       device='cuda:0')


In [20]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, doc_txt[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits_en = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits_en[0:3]:
         print("\t{:.3f}\t{}".format(hit['score'], doc_txt[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits_re = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits_re[0:3]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], doc_txt[hit['corpus_id']].replace("\n", " ")))
    #return hits_re['cross-score'], doc_txt[int(hits_re['corpus_id'])]






In [21]:
search(query = "A 78 year old male presents with frequent stools and melena")


-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.436	solitari peutzjegh polyp jejunum yearold 19year old melena anemia duodenoscopi abnorm bowel xray jejun polyp cm polyp visualis perop bowel endoscopi subsequ surgic remov polyp characterist histolog appear peutzjegh polyp sign peutzjegh characterist mucocutan pigment polyp remov polyp melena recur hemoglobin normal altogeth fulfil peutzjegh solitari jejun peutzjegh polyp polyp  backgroundcas presentationdiscussionconclusionabbreviationscompet interestsauthor contributionscons peutzjegh pj famili occurr gastrointestin hamartomat polyp mucocutan hyperpigment pj histolog peutzjegh polyp pjp pjp pj characterist promin mucocutan pigment pj pjp characterist promin mucocutan pigment pj gastrointestin extraintestin malign gastrointestin hamartomat polyp pj histolog appear interdigit smooth fiber form characterist branch tree arbor polyp pj solitari pjp typic mucocutan pj except 22year old solitari pjp yearold present melena blee

In [25]:
f = open(r'temp1.txt','w')
for q1,q2 in zip(query['QueryID'], query['NERgdd']):
  output1 = {}
  output2 = {}
  output1[q2] = search(q2)
  output2[q1] = search(q2)
  for k, v in output1.items():
    print(k, v)



-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.447	amylas polymorph drosophila melanogast haplotyp frequenc tropic african american    full articl plea pdf 
	0.445	bench bedsid everolimu subependym giant astrocytoma tuber sclerosi optic regener cytotoxin glioma  axon regenerationneuroscienceneurosurgerytuber sclerosi complextarget cytotoxin everolimu subependym giantcel astrocytoma tuber sclerosi pointstun arous optogenet modul locu coeruleu pointslongdist axon regener matur optic contribut oncomodulin camp pten delet pointsgen therapymedi cytotoxin glioma therapeut point  
	0.421	superparamagnet bifunct bisphosphon nanoparticl mri agent osteoporosi nanosystem magnet agent magnet reson mri agent bisphosphon nanoplatform consist superparamagnet fe nanoparticl conjug 15dihydroxy155trisphosphonopentylphosphon dihmbp bisphosphon outer nanoparticl assynthes nanoparticl mri agent adsorpt onto hydroxyapatit mri measur adsorpt bisphosphon nanoparticl hydroxyapatit mri t2 agent 

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f562094a150>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


TypeError: ignored

In [ ]:
top_relevance = queries['summary'].apply(lambda x:search(query = x ))
top_relevance


-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.361	 70 year age schedul joint replac surgeri random tramacet naloxon plu morphin pca morphin pca ass qualiti analgesia postop period primari object determin opioid use combin use oral tramacet naloxon infus periop elderli 70 yr older elect knee hip arthroplasti spinal anesthesia secondari object determin incid va score ≥4 advers effect nausea vomit sedat respiratori depress pruritu confus time independ mobil hypothesi periop naloxon infus tramacet provid adequ analgesia elderli undergo total kneehip arthroplasti associ 80 reduct opioid use reduc opioidinduc side effect
	0.330	 purpos examin variat physic physiolog characterist accord sport age adolesc adult male team handbal th player adolesc n19 age 181±14 yr adult n23 286±57 yr player examin anthropometr characterist somatotyp bodi composit perform physic work capac forceveloc later scapular slide lsst shark skill davi squat jump sj countermov vertic jump without cmj
	0.

TypeError: ignored